## Word2Vec

### Introdução

Em machine learning, o objetivo é empregar o poder computacional para resolver problemas do mundo real. Diferentemente dos humanos, que possuem percepção direta, os computadores operam fundamentalmente com números, convertidos em última instância para sequências de __0's__ e __1's__.

Atualmente, algoritmos processam diversos tipos de dados, incluindo números, imagens, áudios e textos. Para que um computador possa trabalhar com essas informações, é necessário encontrar maneiras de representá-las numericamente, permitindo sua manipulação. No contexto do texto, existem várias técnicas capazes de convertê-lo em informação numérica. Essas técnicas variam desde abordagens mais simples, como a contagem de palavras no modelo _Bag of Words_, até modelos mais sofisticados, como os _Transformers_. Este estudo se concentra na técnica __Word2Vec__.

O __Word2Vec__ consiste em representar palavras em um espaço vetorial de _n-dimensões_, de forma que essa representação capture seu significado semântico. Geralmente, palavras semanticamente relacionadas, como "mulher" e "garota", terão representações vetoriais próximas nesse espaço. Em contraste, a similaridade vetorial entre palavras menos relacionadas, como "mulher" e "homem", será menor. Essa lógica se aplica a outros exemplos de palavras como visualizado abaixo.
> Word2vec é um algoritmo para obter word embeddings treinando uma rede neural rasa (com apenas uma hidden layer) com duas arquiteturas possíveis: CBOW ou Skip-Gram. ([Word Embedding: fazendo o computador entender o significado das palavras](https://medium.com/turing-talks/word-embedding-fazendo-o-computador-entender-o-significado-das-palavras-92fe22745057))

![Representação Vetorial de Palavras em um Plano 3D](https://miro.medium.com/v2/resize:fit:868/0*Cgod6JuBcJyd9GVM)

### Arquiteturas do Word2Vec

Antes de iniciar a exploração, vou importar algumas bibliotecas necessárias que utilizarei para esse projeto (evitar importações desnecessárias ou repetitivas).

In [ ]:
import os
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

Além disso, vou criar algumas funções comuns que vou utilizar nesse notebook python

In [2]:
import requests
import zipfile

# Funtion to download zip file from URL
def download_file_zip(URL: str, file_name: str):
    os.makedirs("data/model/zipfiles", exist_ok=True)

    try:
        # Faz o download
        response = requests.get(URL)
        response.raise_for_status()  # Lança erro se status != 200
        
        # Salva o arquivo
        file_path = os.path.join("data", "model", "zipfiles", f"{file_name}.zip")
        with open(file_path, "wb") as file:
            file.write(response.content)
        return f"data/model/zipfiles/{file_name}.zip"
    
    except Exception as e:
        print(f"Erro: {str(e)}")


def unzip_file(source_path: str, final_path: str):
    with zipfile.ZipFile(source_path, "r") as zip_ref:
        zip_ref.extractall(final_path)
        print("All unzip!")  # Substitua pelo caminho desejado

#### CBOW

A arquitetura CBOW (Continuous Bag-of-Words) tem como objetivo prever uma palavra central com base no contexto das palavras que a cercam. Ela utiliza mais processamento uma vez que ela necessita analisar as palavras ao redor para que ela busque qual melhor palavra que se encaixa naquele contexto.
Na palavra abaixo:
> Eu vou para a __________ estudar com a professora!

O algoritmo analisa as palavras ao redor da palavra buscada e por exemplo, identifica que estudar tem relação com outras palavras como __casa__, __escola__, __biblioteca,__ dentre outras haver com esse contexto. Quando ele analisa as outras palavras da frase ele vê que __professora__ está mais relacionado com __escola__ do que com __biblioteca__ por exemplo e com base nessa ideia ele indica de maneira probabilística que a melhor palavra indicada para esse caso seja __escola__.

Em geral, algumas __vantagens__ que acompanha esse tipo de técnica é que o CBOW converge mais rapidamente do que o Skip-gram, pois precisa prever uma única palavra central a partir de múltiplos contextos, o que torna o problema de aprendizado um pouco mais fácil. Além disso, ele apresenta boa representação para palavras frequentes já que CBOW tende a aprender boas representações para palavras que aparecem com frequência no corpus, pois se beneficia da agregação de informações de múltiplos contextos. Além disso, ele é menos sensível a palavras raras já que como ele utiliza o contexto para prever a palavra central, o impacto de palavras raras no treinamento geral pode ser menor.

##### Explorando um pouco a arquiterura CBOW

Baixando o modelo CBOW treinado para português brasileiro.

In [3]:
URL_CBOW_100D = "http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip"

path_final = download_file_zip(URL_CBOW_100D, "cbow_ptbr_100d")
unzip_file(path_final, "data/model/cbow_ptbr_100d")

All unzip!


In [ ]:
model_cbow = KeyedVectors.load_word2vec_format('data/model/cbow_ptbr_100d/cbow_s100.txt')

Um exemplo de como o modelo representa as palavras, o termo __carro__, é representado nesse modelo de 100 dimensões com o vetor abaixo:

In [5]:
print(model_cbow['carro'])

[ 0.133066 -0.140199 -0.123593 -0.327289  0.527658 -0.093072  0.211601
 -0.080882  0.136657  0.113021 -0.047345  0.193672 -0.343223  0.140333
  0.114996  0.302133 -0.276257 -0.019003 -0.170397  0.006797  0.26281
  0.48911  -0.259811  0.565083  0.19642   0.107387 -0.01361  -0.128821
 -0.142525  0.145407 -0.143249 -0.17207   0.019843 -0.181988  0.277796
  0.159842  0.233585 -0.211342  0.094705  0.132454  0.263601  0.212976
 -0.016437 -0.112659  0.501575 -0.267115 -0.128304  0.054397 -0.240181
 -0.12104   0.141273 -0.506544  0.21905  -0.302853 -0.463127 -0.002334
  0.284288  0.319843  0.468935 -0.363241 -0.191774  0.024671  0.163243
 -0.099187 -0.025341  0.094963  0.166238  0.149067  0.306253 -0.201536
 -0.327082  0.035873 -0.096247  0.086678  0.103305 -0.249064 -0.259611
 -0.472722 -0.485474 -0.344718 -0.216763 -0.188939  0.387807  0.095012
  0.148751  0.078313  0.205438 -0.076185 -0.080932 -0.19896  -0.246814
  0.007546 -0.12768  -0.071645 -0.040573  0.210866  0.240887 -0.087581
  0.425

Assim como o termo __veículo__ é representado pelo vetor:

In [6]:
print(model_cbow['veículo'])

[ 0.006231 -0.054839 -0.211529 -0.261954  0.585775 -0.118988  0.226396
 -0.039966  0.015274  0.131516 -0.105483  0.320252 -0.271816  0.278673
  0.09556   0.303267 -0.36492   0.106018 -0.292437 -0.021026  0.239087
  0.499897 -0.16466   0.511368  0.062144  0.116286 -0.064749 -0.146283
 -0.063641  0.15635   0.086804 -0.109098  0.152468 -0.362194  0.28604
 -0.037003  0.216354 -0.076125  0.099117  0.155871  0.132667 -0.077328
  0.054892 -0.163485  0.459812 -0.194224 -0.076284 -0.062852 -0.306793
 -0.091259  0.281505 -0.49223   0.091281 -0.314142 -0.366766 -0.130913
  0.244339  0.316981  0.460022 -0.49401  -0.281279 -0.065548  0.06151
 -0.026801 -0.04999   0.067487  0.05858  -0.03238   0.201022 -0.028652
 -0.239572  0.088996 -0.062438  0.119585  0.050447 -0.350612 -0.166851
 -0.528883 -0.618139 -0.466495 -0.257315 -0.135686  0.347661  0.087382
  0.081486  0.080525  0.222738 -0.207515  0.020843 -0.211405 -0.277013
 -0.06448  -0.21568  -0.00472  -0.156896  0.110703  0.221783  0.037422
  0.3399

##### Explorando o modelo

Uma vez com esse modelo carregado, pode-se utilizar algumas funções da biblioteca para exploração:

In [10]:
# Encontrar as 10 palavras mais similares do modelo
lista_palavras = model_cbow.most_similar('carro', topn=10)
lista_mais_similares_carro = pd.DataFrame(lista_palavras, columns=['Palavra', 'Similaridade'])
lista_mais_similares_carro.head(10)

,Palavra,Similaridade
0,veículo,0.924637
1,caminhão,0.886713
2,jipe,0.846801
3,avião,0.831295
4,barco,0.813364
5,parabrisa,0.799635
6,elevador,0.793397
7,cofre,0.788872
8,carrinho,0.787174
9,passageiro,0.785835


E qual seria a similaridade entre as palavas __mulher__ e __rainha__?

In [12]:
print(f"A similaridade entre as palavras MULHER e RAINHA é {model_cbow.similarity('mulher', 'rainha')}")

A similaridade entre as palavras MULHER e RAINHA é 0.4940199851989746


Descobrindo a palavra dentre um grupo que menos tem relação com as outras do grupo.

In [15]:
lista_palavras = ["maçã", "banana", "laranja", "cachorro"]
palavra_menos_similar = model_cbow.doesnt_match(lista_palavras)
print(f"A palavra do grupo {lista_palavras} que menos está relacionado é {palavra_menos_similar.upper()}!")

A palavra do grupo ['maçã', 'banana', 'laranja', 'cachorro'] que menos está relacionado é CACHORRO!


#### SKIPGRAM

A arquitetura Skip-Gram (Continuous Skip-Gram) tem como objetivo prever as palavras de contexto a partir da palavra central. Ela utiliza mais processamento, pois precisa prever múltiplas palavras de contexto para cada palavra de entrada, calculando probabilidades para cada termo no vocabulário.

Na palavra abaixo:

> Eu vou para a escola estudar com a ______!

O algoritmo toma como entrada a palavra central (por exemplo, estudar) e, a partir dela, tenta prever quais palavras costumam aparecer ao seu redor. Nesse contexto, ele identifica que “estudar” está associado a termos como “casa”, “escola”, “biblioteca”, “professora” etc. Ao processar a frase acima, o Skip-Gram calcula a probabilidade de cada uma dessas palavras aparecer no lugar do blank e, de forma probabilística, indica qual contexto é mais provável naquele ponto (por exemplo, “professora”).

Em geral, algumas vantagens que acompanham esse tipo de técnica são uma melhora para palavras raras uma vez que como o modelo prevê múltiplos contextos para cada palavra central, o Skip-Gram aprende representações de alta qualidade mesmo para termos pouco frequentes no corpus. Além disso, essa arquitetura captura nuances dos diferentes contextos em que uma palavra aparece, pois cada ocorrência contribui para ajustar o vetor central.

##### Explorando um pouco a arquiterura CBOW

Baixando o modelo SKIPGRAM para o português brasileiro.

In [16]:
URL_SKIPGRAM_100D = "http://143.107.183.175:22980/download.php?file=embeddings/word2vec/skip_s100.zip"

path_final = download_file_zip(URL_SKIPGRAM_100D, "skipgram_ptbr_100d")
unzip_file(path_final, "data/model/skipgram_ptbr_100d")

All unzip!


In [17]:
model_skipgram = KeyedVectors.load_word2vec_format("data/model/skipgram_ptbr_100d/skip_s100.txt")

Para os mesmos termos anteriores, agora tem-se a aplicação com a arquitetura SKIPGRAM

In [18]:
print(model_skipgram['carro']) # Vetor que representa o termo 'carro'

[-0.03052  -0.179981  0.318288 -0.420714  0.240731 -0.02601  -0.077635
  0.228754  0.129099  0.09437   0.358149 -0.419461 -0.296144  0.243431
  0.209637  0.153136 -0.01747   0.005728  0.053692  0.008757  0.224109
  0.214447  0.266326 -0.410923  0.111632 -0.412724 -0.322653  0.191501
  0.161768  0.308219  0.261416  0.05929  -0.327802 -0.108396 -0.067312
 -0.374585 -0.292027 -0.032074 -0.088213  0.164639 -0.187844 -0.046748
  0.325778 -0.272374 -0.241957 -0.190494 -0.068726  0.182147  0.314323
  0.18177  -0.160564 -0.178233  0.291935  0.189777 -0.269338 -0.047054
 -0.215373  0.008573  0.045078  0.130132 -0.080728 -0.260449 -0.011681
  0.119967 -0.219406 -0.17135   0.226871  0.020841  0.084167  0.065814
  0.258219  0.439529  0.006809 -0.115177 -0.103107  0.054834  0.698577
 -0.161539 -0.18658  -0.122717 -0.281636  0.181422 -0.076578 -0.111674
 -0.141989 -0.874926  0.28169   0.117719  0.267188 -0.063896 -0.406935
  0.396001 -0.41795  -0.229335 -0.072306  0.165195 -0.199483  0.244025
  0.57

In [19]:
print(model_skipgram['veículo']) # Vetor que representa o termo 'veículo'

[-0.048543 -0.146925  0.14268  -0.550171  0.184191 -0.010326 -0.039762
  0.247268  0.05561   0.059013  0.329475 -0.339056 -0.226146  0.126701
  0.137502  0.316843 -0.128018  0.136733  0.091175  0.024942  0.200224
  0.113542  0.343585 -0.493607  0.093524 -0.587963 -0.506089  0.255164
  0.195453  0.359028  0.230092  0.237051 -0.291982 -0.073544  0.122711
 -0.341275 -0.232137  0.106199 -0.015645  0.009579 -0.396079 -0.109282
  0.332166 -0.179727 -0.283452 -0.11684  -0.111095  0.148477  0.256435
  0.197933 -0.119023 -0.160942  0.14026   0.232627 -0.301246  0.065198
 -0.26916  -0.151104  0.066499 -0.036433 -0.243753 -0.116231  0.005079
  0.222212 -0.14063  -0.028491  0.187754  0.037357  0.018899  0.132222
  0.129842  0.568389 -0.124462 -0.009716 -0.001952  0.224699  0.786892
 -0.363619 -0.2902   -0.135668 -0.362552  0.076921  0.031073  0.008528
 -0.025896 -0.82875   0.220282  0.149317  0.455413  0.007924 -0.505413
  0.385133 -0.314584 -0.337715 -0.015384  0.178757 -0.126213  0.26852
  0.415

#### Explorando o Modelo

Agora, com a outra arquitetura, seus valores são diferentes pois seu treinamento foi feito com outra técnica.

In [20]:
# Encontrar as 10 palavras mais similares do modelo
lista_palavras = model_skipgram.most_similar('carro', topn=10)
lista_mais_similares_carro = pd.DataFrame(lista_palavras, columns=['Palavra', 'Similaridade'])
lista_mais_similares_carro.head(10)

,Palavra,Similaridade
0,veículo,0.926959
1,caminhão,0.895195
2,passageiro,0.849853
3,jipe,0.823606
4,motorista,0.819495
5,trator,0.814430
6,furgão,0.805145
7,táxi,0.792833
8,motociclista,0.773929
9,ciclomotor,0.770150


Medindo a similaridade entre os termos __mulher__ e __rainha__.

In [22]:
print(f"A similaridade entre as palavras MULHER e RAINHA é {model_skipgram.similarity('mulher', 'rainha')}")

A similaridade entre as palavras MULHER e RAINHA é 0.5242229700088501


A mesma prática para descobrir qual o termo menos relacionado. Nesse caso, indepedentemente da arquitetura, a eficiência se mantém.

In [23]:
lista_palavras = ["maçã", "banana", "laranja", "cachorro"]
palavra_menos_similar = model_skipgram.doesnt_match(lista_palavras)
print(f"A palavra do grupo {lista_palavras} que menos está relacionado é {palavra_menos_similar.upper()}!")

A palavra do grupo ['maçã', 'banana', 'laranja', 'cachorro'] que menos está relacionado é CACHORRO!


### Referências

- [Word2Vec e sua importância na etapa de pré-processamento](https://medium.com/@everton.tomalok/word2vec-e-sua-importância-na-etapa-de-pré-processamento-d0813acfc8ab)
- [Word Embedding: fazendo o computador entender o significado das palavras](https://medium.com/turing-talks/word-embedding-fazendo-o-computador-entender-o-significado-das-palavras-92fe22745057)
- [What Is Word2Vec and How Does It Work?](https://swimm.io/learn/large-language-models/what-is-word2vec-and-how-does-it-work)